In [ ]:
from comet_ml import Experiment
import GPUtil
import torch

import os
import codecarbon
from iesta.data.iesta_data import IESTAData, LABELS
from iesta.data.huggingface_loader import IESTAHuggingFace
from nlpaf.transformers.text_classification import TextClassification
from dotenv import load_dotenv, find_dotenv
import argparse
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
import re

import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport


def _apply_no_punc(row):
    row["text_no_punc"] = re.sub(r"[^\w\s]", "", row["text"])
    return row


def profile_datasets(
    ds_name: str = "debateorg_w_effect_for_liberal", lower: bool = False
):
    dataset_dict: DatasetDict = load_dataset(f"notaphoenix/{ds_name}")
    dfs = []
    for split, ds in dataset_dict.items():
        print(split)
        dfs.append(ds.to_pandas())

    df = pd.concat(dfs)
    len(df)
    # profile = ProfileReport(df, title="Profiling Report")
    # df["text_low"] = df["text"].str.lower()
    dissmiss_arr = []
    with open("../data/dismiss_text.txt", "r") as dismissedf:
        dissmiss_arr = list(
            pd.Series(dismissedf.read().splitlines()).str.lower()
        )
        dissmiss_arr = list(
            set([re.sub(r"[^\w\s]", "", x) for x in dissmiss_arr])
        )

    df = df.apply(_apply_no_punc, axis=1)
    df = df[~df["text_no_punc"].str.lower().isin(dissmiss_arr)]
    if lower:
        df["text_low"] = df["text"].str.lower()
        profile = ProfileReport(df[["text_low"]], title="Profiling Report")
        profile.to_file(f"../data/profilers/{ds_name}_low.html")
    else:
        profile = ProfileReport(df[["text"]], title="Profiling Report")
        profile.to_file(f"../data/profilers/{ds_name}.html")
    return df, profile

In [ ]:
# This is typically needed once per notebook
# %matplotlib inline
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from nltk.tokenize import word_tokenize

sns.set(style="darkgrid")
import re

In [ ]:
def plot_datasets(
    ds_name: str = "debateorg_w_effect_for_liberal", lower: bool = False
):
    dataset_dict: DatasetDict = load_dataset(f"notaphoenix/{ds_name}")
    dfs = []
    for split, ds in dataset_dict.items():
        print(split)
        dfs.append(ds.to_pandas())

    df = pd.concat(dfs)
    len(df)
    # profile = ProfileReport(df, title="Profiling Report")
    # df["text_low"] = df["text"].str.lower()
    dissmiss_arr = []
    with open("../data/dismiss_text.txt", "r") as dismissedf:
        dissmiss_arr = list(
            pd.Series(dismissedf.read().splitlines()).str.lower()
        )
        dissmiss_arr = list(
            set([re.sub(r"[^\w\s]", "", x) for x in dissmiss_arr])
        )

    df = df.apply(_apply_no_punc, axis=1)
    df = df[~df["text_no_punc"].str.lower().isin(dissmiss_arr)]

    df["num_tokens"] = df["text"].apply(lambda x: len(word_tokenize(x)))
    df["num_chars"] = df["text"].apply(lambda x: len(x))

    # df = df[df['num_tokens']>2]
    # df = df[df['num_tokens']<=1600]
    sns.set(style="darkgrid")
    fig, axes = plt.subplots(figsize=(10, 7))
    # Plot histogram
    color = "olive"
    for lbl, df_ in df.groupby(["label"]):
        sns.histplot(
            df_["num_chars"], bins=50, color=color, label=lbl, stat="percent"
        )
        color = "skyblue"

    plt.title(f"Histogram of Token Counts {ds_name} ")
    plt.xlabel("Number of Tokens")
    plt.ylabel("Frequency")
    plt.legend()
    plt.show()
    return df

In [ ]:
dataset_dict: DatasetDict = load_dataset(
    f"notaphoenix/debateorg_w_effect_for_liberal_subset"
)

dataset_dict = dataset_dict.remove_columns(
    ["author", "original_text", "category", "round", "debate_id", "idx"],
)

In [ ]:
dataset_dict["validation"]

In [ ]:
for lbl, df_ in lib.groupby(["label"]):
    print(lbl)
    prev = 0
    for x in range(5, 2001, 295):
        print(
            f"between {prev} and {x} (inclusive): ",
            len(df_[(df_["num_tokens"] > prev) & (df_["num_tokens"] <= x)]),
        )
        prev = x
    print(f"<= 600: ", len(df_[(df_["num_tokens"] <= 600)]))
    print(f"> 600: ", len(df_[(df_["num_tokens"] > 600)]))
for lbl, df_ in lib.groupby(["label"]):
    print(lbl)
    prev = 0
    for x in range(20, 10000, 500):
        print(
            f"between {prev} and {x} (inclusive): ",
            len(df_[(df_["num_tokens"] > prev) & (df_["num_tokens"] <= x)]),
        )
        prev = x

In [ ]:
dataset_names = [
    "debateorg_w_effect_for_liberal_subset",
    "debateorg_w_effect_for_conservative_subset",
    "debateorg_w_effect_for_liberal",
    "debateorg_w_effect_for_conservative",
]


for ds_name in dataset_names:
    # profile_datasets(ds_name, lower=True)
    plot_datasets(ds_name)
    break

In [ ]:
from iesta.data.iesta_data import IESTAData, LABELS
from iesta.data.huggingface_loader import IESTAHuggingFace

conservative_data_obj = IESTAData(
    ideology="conservative",
    keep_labels=LABELS.EFF_INEFF,
)
liberal_data_obj = IESTAData(
    ideology="liberal",
    keep_labels=LABELS.EFF_INEFF,
)

In [ ]:
cons_df, df = conservative_data_obj.split_iesta_dataset_by_debate(
    True, profile=True
)

In [ ]:
lib_df, df = liberal_data_obj.split_iesta_dataset_by_debate(True, profile=True)

In [ ]:
lib_df

In [ ]:
df[0]

In [ ]:
df_ = df[0]

In [ ]:
df_.index.name = "idx"
df_[
    [
        "id",
        "debate_id",
        "p_name",
        "top_effect",
        "category",
        "round",
        "argument",
        "cleaned_text",
    ]
]

In [ ]:
df_.to_parquet("temp.parquet")

In [ ]:
pip show iesta

In [ ]:
%pip -q install -U ../



In [ ]:
from iesta.data.iesta_data import IESTAData, LABELS
from iesta.data.huggingface_loader import IESTAHuggingFace

ideology = "liberal"
data_obj = IESTAData(
    ideology=ideology,
    keep_labels=LABELS.EFF_INEFF,
)
hf = IESTAHuggingFace(data_obj, reload_preprocess=False)
style_eval_data = hf.upload_w_labels(
    is_for_style_classifier=False, force_reload=False
)

In [ ]:
from datasets.dataset_dict import Dataset, DatasetDict


# notaphoenix/debateorg_w_effect_for_liberal

In [ ]:
from datasets import load_dataset

ds = load_dataset(
    "notaphoenix/debateorg_w_effect_for_liberal", use_auth_token=True
)

In [ ]:
ds

In [ ]:
import pandas as pd


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
counts_dict = {"liberal": None, "conservative": None}
for ideology in ["liberal", "conservative"]:
    print(f"\n###### {ideology.capitalize()} ###### ")
    ds = load_dataset(
        f"notaphoenix/debateorg_w_effect_for_{ideology}", use_auth_token=True
    )
    ds_2 = load_dataset(
        f"notaphoenix/debateorg_w_effect_for_{ideology}_subset", use_auth_token=True
    )
    
    for split in ["training", "test", "validation"]:
        print(f"\n****** {split.capitalize()} ***** ")
        dd = ds[split]
    
        print()
        df = dd.to_pandas()["label"].value_counts().to_frame() + ds_2[split].to_pandas()["label"].value_counts().to_frame()
        df = df.rename(columns={"label": split})
        if counts_dict[ideology] is None:
            counts_dict[ideology] = df
        else:
            counts_dict[ideology] = counts_dict[ideology].merge(df, right_index=True, left_index=True, )
        
        # liberal 36,880,  1,845
        # liberal 36,880,  2,054
    counts_dict[ideology] = counts_dict[ideology].T.rename(columns={0: f"Ineffective_{ideology}",
                                                                    1: f"effective_{ideology}"})

all_counts_df = counts_dict["conservative"].merge(counts_dict["liberal"], right_index=True, left_index=True, )


In [ ]:
all_counts_df.to_csv("data/all_counts_splits.csv")

In [ ]:
ds_2[split].to_pandas()["label"].value_counts().to_frame() +dd.to_pandas()["label"].value_counts().to_frame()

In [ ]:
ds_2[split].to_pandas()["label"].value_counts().to_frame() 

In [ ]:
from glob import glob
import pandas as pd
import itertools
ideology = "liberal"
idx_per_exp_dict = {}
for path in glob(f"data/llms_out/new/{ideology}_*.jsonl"):
    df = pd.read_json(path_or_buf=path, lines=True)
    df = df.drop_duplicates(subset=["idx"], keep="last")
    idx_per_exp_dict[path] = df['idx'].unique()

    #print(f"Path {path}: {len(df['idx'].unique())}")

exps = idx_per_exp_dict.keys()
for n in range(2, len(exps)+1):
    print(f"############## {n} ###################")
    for subset in itertools.combinations(exps, n):
        res = []
        for e in subset: 
            if len(res )== 0:
                res  = idx_per_exp_dict[e]
            else:
                res = [x for x in res if x in idx_per_exp_dict[e]]
        print(f"{subset}: {len(res)}")   

In [49]:
ideology = "conservative"
path = f"data/llms_out/new/{ideology}_llamav2_1shot.jsonl"


df = pd.read_json(path_or_buf=path, lines=True)
df = df.drop_duplicates(subset=["idx"], keep="last")
df[["idx"]].to_csv(f"data/out/{ideology}_idx.csv", index=False)